In [1]:
import os ; import pathlib as pl ; import pandas as pd ; import numpy as np ;  import matplotlib.pyplot as plt ; from pathlib import Path  
from flopy.plot.styles import styles ; import shapefile as shp ; import matplotlib as mpl ; import flopy ; import flopy.utils.binaryfile as bf 
from modflow_devtools.misc import get_env, timed ; write=get_env("WRITE", True);run=get_env("RUN", True) ; from flopy.utils.lgrutil import Lgr 
sim_ws    = Path("./24it_LGR")                       ; sim_ws.mkdir(exist_ok=True)     ; sim_name = modelname= 'ex-gwf-lgrv-gc'
exe_name  = 'E:\\15_REPOS\\00_BETAMI\\bin\\mf6.exe'  ; workspace = figs_path = sim_ws       

In [2]:
length_units = "meters" ; time_units = "seconds"    ; k11_str = "5.e-07, 1.e-06, 5.e-05" ; nouter = 50 ; ninner = 100 ; hclose = 1e-6 ; rclose = 100.0
nper = 1 ;nlay = 25 ;nrow = 183 ;ncol = 147         ; nlaygc = 9 ;nrowcg = 61 ;ncolcg = 49 ;delr = 35 ;delc = 25 ;delv = 5 ;delrgc = 105 
delcgc = 75 ;delvgc = 15 ; top_str = "variable"     ; botm_str = "30 to -90" ; icelltype = 0 ; recharge = 0.0000000011098 ;

In [3]:
path = 'E:\\15_REPOS\\00_BETAMI\\bin\\data_modflow6-examples\\ex-gwf-lgrv'                 ; top = np.loadtxt(os.path.join(path,"top.dat"))
dt = [("k",int),("i",int),("j",int) ,("stage",float),("conductance",float),("rbot",float)] ; rivdat=np.loadtxt(os.path.join(path,"riv.dat"), dtype=dt)
ikzone = np.empty((nlay, nrow, ncol),dtype=float)
for k in range(nlay):                        
    fname = f"ikzone{k + 1}.dat"    ;f_ikzone = os.path.join(path, fname)                   ; ikzone[k, :, :] = np.loadtxt(f_ikzone)
a1 = pd.DataFrame(ikzone[24,:,:]) ; a1.to_csv (sim_ws/ "03_ikzone_xy.csv")                  ; print('ikzone',ikzone.shape)   # 01 02 03 *.csv
a1 = pd.DataFrame(top)    ; a1.to_csv (sim_ws/ "01_top.csv") ; a1 = pd.DataFrame(rivdat)    ; a1.to_csv (sim_ws/ "02_riv.csv")

ikzone (25, 183, 147)


In [5]:
perlen = [1.0]    ; nstp = [1] ; tsmult = [1.0]             ; tdis_ds = list(zip(perlen, nstp, tsmult))
rivdat["k"] -= 1  ; rivdat["i"] -= 1  ; rivdat["j"] -= 1    ; riv_spd = [[(k, i, j), stage, cond, rbot] for k, i, j, stage, cond, rbot in rivdat]
botm = [30 - k * delv for k in range(nlay)]                 ; botm = np.array(botm)

k11_values = [float(value) for value in k11_str.split(",")] ; k11 = np.zeros((nlay, nrow, ncol), dtype=float)
for i, kval in enumerate(k11_values):                         k11 = np.where(ikzone == i + 1, kval, k11)

xmin = 0.0 ; xmax = ncol * delr          ; ymin = 0.0 ; ymax = nrow * delc   ; parent_domain = [xmin, xmax, ymin, ymax]
child_domain = [xmin + 15 * 3 * delr   , xmin + 41 * 3 * delr                , ymax - 49 * 3 * delc,ymax - 19 * 3 * delc]    # child_domain
print('parent_domain', parent_domain, '    child', child_domain, "    layers",len(botm) , '    riv_spd = 02_riv.csv')
ncppl = [1, 3, 3, 3, 3, 3, 3, 3, 3]  ;nlayc = len(ncppl) ;layer_index = [ncppl[0] - 1] ;print('layer_index', layer_index, 'k11',k11[0].shape)
idomainp = np.ones((nlay, nrow, ncol), dtype=int)  ; xorigin = xmin ; yorigin = ymin 
a1=pd.DataFrame(k11[20]) ;a1.to_csv(sim_ws/"04_k11_3x.csv")

parent_domain [0.0, 5145, 0.0, 4575]     child [1575.0, 4305.0, 900, 3150]     layers 25     riv_spd = 02_riv.csv
layer_index [0] k11 (183, 147)


In [6]:
k11_values = [float(value) for value in k11_str.split(",")] ; k11_values

k11 = np.zeros((nlay, nrow, ncol), dtype=float)
for i, kval in enumerate(k11_values):                k11 = np.where(ikzone == i + 1, kval, k11)  # link between ikzone
print('nlayc',nlayc) 
top

nlayc 9


array([[76.59 , 77.193, 77.795, ..., 32.582, 32.068, 31.554],
       [76.126, 76.61 , 77.094, ..., 32.732, 32.253, 31.774],
       [75.662, 76.027, 76.392, ..., 32.882, 32.438, 31.995],
       ...,
       [60.739, 62.666, 64.594, ..., 58.634, 57.358, 56.082],
       [60.555, 63.01 , 65.465, ..., 58.837, 57.389, 55.941],
       [60.371, 63.354, 66.337, ..., 59.04 , 57.42 , 55.8  ]])